In [ ]:
import os
os.environ["HF_HOME"] = "pathtocache"

import torch
from PIL import Image
from PIL import Image, ImageOps
import math



from lavis.models import load_model_and_preprocess
from lavis.processors import load_processor
from transformers import AutoTokenizer, LlamaForCausalLM
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer




import csv
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score

from transformers import AutoProcessor, LlavaForConditionalGeneration
from transformers import LlavaNextProcessor, LlavaNextForConditionalGeneration
import requests
import matplotlib.pyplot as plt
from transformers import pipeline
from transformers import T5Tokenizer, T5ForConditionalGeneration


In [ ]:

def read_csv(file_path):
    return pd.read_csv(file_path, encoding='latin-1')



def list_files(directory):
    return os.listdir(directory)

def list_folders_with_numbers(directory):
    folders_with_numbers = []
    folders = [folder for folder in os.listdir(directory) if os.path.isdir(os.path.join(directory, folder))]
    for folder in folders:
        if any(char.isdigit() for char in folder):
            folders_with_numbers.append(folder)
    folders_with_numbers.sort(key=lambda x: int(''.join(filter(str.isdigit, x))))
    return folders_with_numbers


def select_rows(csv_data, files_directory):
    selected_rows = []
    files = list_folders_with_numbers(files_directory)

    for filename in files:
        try:
            file_index = int(filename.split('-')[1])
            selected_rows.append(csv_data.iloc[file_index-2])


        except (ValueError, IndexError):
            pass

    return selected_rows

if __name__ == "__main__":
    csv_file_path = '<pathftofile>'
    files_directory = '<pathtoimagefiles>'

    csv_data = read_csv(csv_file_path)
    selected_rows = select_rows(csv_data, files_directory)


    custom_index = list(range(0,len(selected_rows)))
    df_syn = pd.DataFrame(selected_rows, columns=['Questions', 'answers'],index=custom_index)

In [ ]:
directory_name = f"<pathtodirectory>"


if not os.path.exists(directory_name):

    os.makedirs(directory_name)
    print("Directory created:", directory_name)
else:
    print("Directory already exists:", directory_name)



# Text-only LLMs

## Flan-T5

In [ ]:
model_name = "google/flan-t5-small"



model = AutoModelForSeq2SeqLM.from_pretrained(model_name, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
eval_prompt_1 = df_syn['Questions'].tolist()


test_lst = []
for i in eval_prompt_1:
    item = "Answer with the option's letter from the given choices directly and don't provide extra choices or explanations." + '\n' + i + '\n' + "Answer is:"

    inputs = tokenizer(item, return_tensors="pt").to('cuda')
    generate_ids = model.generate(**inputs, max_new_tokens=100)
    test = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
    test_lst.append(test)

df_llama_2_7b = pd.DataFrame({'col1': test_lst})

## flan-alpaca-gpt4-xl

In [ ]:
model_name = "declare-lab/flan-alpaca-gpt4-xl"


model = pipeline(model=model_name, device_map= 'auto')



eval_prompt_1 = df_syn['Questions'].tolist()


test_lst = []
for i in eval_prompt_1:
    item = "Answer with the option's letter from the given choices directly and don't provide extra choices or explanations." + '\n' + i + '\n' + "Answer is:"

    with torch.no_grad():
        test = model(item, max_length=128, do_sample=True)
        test_lst.append(test)

df_llama_2_7b = pd.DataFrame({'col1': test_lst})
df_llama_2_7b.loc[0, 'col1']

## LLaMA-2

In [ ]:
model_name = "meta-llama/Llama-2-70b-chat-hf"


access_token = "<access_token>"

cache_dir = "pathtocache"

bnb_config = BitsAndBytesConfig(
load_in_4bit=True,
bnb_4bit_use_double_quant=True,
bnb_4bit_quant_type="nf4",
bnb_4bit_compute_dtype=torch.bfloat16
)

model = LlamaForCausalLM.from_pretrained(model_name, device_map="auto",
                                         quantization_config=bnb_config,
                                         use_auth_token=access_token,
                                         torch_dtype=torch.float16,
                                         cache_dir = cache_dir,
                                         attn_implementation="flash_attention_2"
                                        )
tokenizer = AutoTokenizer.from_pretrained(model_name,
                                          use_fast=True,
                                          use_auth_token=access_token,
                                          cache_dir = cache_dir
                                         )

In [ ]:
eval_prompt_1 = df_syn['Questions'].tolist()


test_lst = []
for i in eval_prompt_1:
    item = "Answer with the option's letter from the given choices directly and don't provide extra choices or explanations." + '\n' + i + '\n' + "Answer is:"

    inputs = tokenizer(item, return_tensors="pt").to('cuda')
    generate_ids = model.generate(**inputs, max_new_tokens=100)
    test = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
    test_lst.append(test)

df_llama_2_7b = pd.DataFrame({'col1': test_lst})

## LLaMA-3

In [ ]:

model_id = "meta-llama/Meta-Llama-3-8B-instruct"



pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={
        "torch_dtype": torch.float16,
        "quantization_config": {"load_in_4bit": True},
        "low_cpu_mem_usage": True,
    },
)


messages = [
    {"role": "system", "content": "Answer following question by choosing an option! Please ONLY provide letter choice as answer to question. "},
    {"role": "user", "content": "put the question here"},
]


df = pd.read_csv("<csv_file>")

column_mapping = {'Questions': 'content'}



index_to_update = 1

update_message = []

for i in range(len(df)):
    for df_col, dict_key in column_mapping.items():
            messages[index_to_update][dict_key] = df[df_col].values[i]

            prompt = pipeline.tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
            )

            terminators = [
            pipeline.tokenizer.eos_token_id,
            pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
            ]

            outputs = pipeline(
            prompt,
            max_new_tokens=256,
            eos_token_id=terminators,
            do_sample=True,
            temperature=0.6,
            top_p=0.9

            )


            temp_dict = {
                "question": df[df_col].values[i],
                "generated_answer": outputs[0]["generated_text"][len(prompt):]
            }
            update_message.append(temp_dict)


second_part = model_id.split('/')[1]
file_name = second_part + "_affordance.csv"


directory_name = f"pathtodirctory{second_part}"


if not os.path.exists(directory_name):
    os.makedirs(directory_name)
    print("Directory created:", directory_name)
else:
    print("Directory already exists:", directory_name)


full_path = f"{directory_name}/{file_name}"

dataframe_tmp = pd.DataFrame.from_dict(update_message, orient='columns')


### flan-alpaca-gpt4-xl cleaning

In [ ]:
def formating(dataframe):
    listofwords = []

    df_conversion = dataframe.copy()


    for word in df_conversion["col1"].values.tolist():

        before_keyword, keyword, after_keyword = word[0]['generated_text'].partition(')')
        listofwords.append(before_keyword.lower())

    df_syn.loc[:,'Generated_Answers'] = listofwords
    print(np.unique(df_syn.iloc[:,2]), np.unique(df_syn.iloc[:,1]))

    return df_syn

final_df = formating(df_llama_2_7b)

### LLaMA-2 cleaning

In [ ]:
def formating(dataframe):
    listofwords = []

    df_conversion = dataframe.copy()

    for word in df_conversion['col1'].values.tolist():

        keyword = 'Answer is:'
        before_keyword, keyword, after_keyword = word.partition(keyword)
        listofwords.append(after_keyword)



    listofwords_ = []
    for words_ in listofwords:
        keyword = ')'
        before_keyword, keyword, after_keyword = words_.partition(keyword)
        listofwords_.append(before_keyword)

    df_syn.loc[:,'Generated_Answers'] = listofwords_
    print(np.unique(df_syn.iloc[:,2]), np.unique(df_syn.iloc[:,1]))

    return df_syn



final_df = formating(dataframe_tmp)

### LLaMA-3 cleaning

In [ ]:
model_name = "meta-llama/Meta-Llama-3-8B-instruct"

def formating(dataframe):
    listofwords = []

    df_conversion = dataframe.copy()

    listofwords_ = []
    for words_ in df_conversion["generated_answer"].values.tolist():
        keyword = ')'
        before_keyword, keyword, after_keyword = words_.partition(keyword)
        listofwords_.append(before_keyword)


    df_syn.loc[:,'Generated_Answers'] = listofwords_
    print(np.unique(df_syn.iloc[:,2]), np.unique(df_syn.iloc[:,1]))

    return df_syn

final_df = formating(dataframe_tmp)

## Flan-T5 cleaning

In [ ]:
def formating(dataframe):
    listofwords = []

    df_conversion = dataframe.copy()

    for word in df_conversion['Generated_Answers'].values.tolist():

        keyword = ')'
        before_keyword, keyword, after_keyword = word.partition(keyword)
        listofwords.append(before_keyword)

    df_syn.loc[:,'Generated_Answers'] = listofwords
    return df_syn

final_df = formating(df_tmp)

In [ ]:
def formatting_text_only(df):
    df.iloc[:,2] = df.iloc[:,2].replace('\n','', regex=True)
    df.iloc[:,2] = df.iloc[:,2].replace('\n\n','', regex=True)
    df.iloc[:,2] = df.iloc[:,2].replace(' a','a', regex=True)
    df.iloc[:,2] = df.iloc[:,2].replace([' \(b'],'b', regex=True)
    df.iloc[:,2] = df.iloc[:,2].replace([' c', 'c ', ' \(c'],'c', regex=True)
    df.iloc[:,2] = df.iloc[:,2].replace(' d','d', regex=True)
    df.iloc[:,2] = df.iloc[:,2].replace(' \(d','d', regex=True)
    df.iloc[:,2] = df.iloc[:,2].replace(' \(a','a', regex=True)
    df.iloc[:,2] = df.iloc[:,2].replace([' b',' b'],'b', regex=True)
    df.iloc[:,2] = df.iloc[:,2].replace(' e','e', regex=True)
    df.iloc[:,2] = df.iloc[:,2].replace(' f','f', regex=True)
    df.iloc[:,2] = df.iloc[:,2].replace(' A','a', regex=True)
    print(np.unique(df.iloc[:,2]), np.unique(df.iloc[:,1]))
    return df

df_final = formatting_text_only(final_df)

## Unifiedqa2 (large-1363200)


In [ ]:

model_name = "allenai/unifiedqa-v2-t5-large-1363200"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

def run_model(input_string, **generator_args):
    input_string = "Answer with the option's letter from the given choices directly.\n" + input_string
    input_ids = tokenizer.encode(input_string, return_tensors="pt")
    res = model.generate(input_ids, **generator_args)
    return tokenizer.batch_decode(res, skip_special_tokens=True)

df_list = []
for item in df_syn['Questions'].tolist():
    df_list.append(run_model(item))
df_append = pd.DataFrame(df_list, columns=['Generated_Answers'])
concat = pd.concat([df_syn, df_append], axis=1)
concat.head()

In [ ]:
def extract_question_and_options(question_text):
    question_options = question_text.split('\n')
    question = question_options[0].strip()
    options = [option.strip() for option in question_options[1:] if option.strip()]
    return question, options

def read_csv():

    df = df_syn

    questions = []
    option_lists = []
    for index, row in df.iterrows():
        question_text = str(row[0])

        question, options = extract_question_and_options(question_text)
        questions.append(question)
        option_lists.append(options)

    return questions, option_lists



questions, options = read_csv()


In [ ]:
combined_list = []

options_lower = [[x.lower() for x in inner_list] for inner_list in options]

combined_list = []

for i, row in concat["Generated_Answers"].items():
    row_lower = row.lower()
    matched = False
    for j, item in enumerate(options[i]):
        if row_lower in item.lower():
            combined_list.append((i, item, "Matched"))
            matched = True
            break
    if not matched:
        combined_list.append((i, row, "Unmatched"))

In [ ]:
df_match_unmatch = pd.DataFrame(combined_list, columns=['Index', 'Item', 'Status'])
concat_final = pd.concat([df_match_unmatch, df_syn], axis=1)


In [ ]:
def formating_unifiedqa(dataframe):
    listofwords = []

    df_conversion = dataframe.copy()

    listofwords_ = []
    for words_ in df_conversion["Item"].tolist():
        keyword = ')'
        before_keyword, keyword, after_keyword = words_.partition(keyword)
        listofwords_.append(before_keyword)

    concat_final.loc[:,'Generated_Answers_final'] = listofwords_
    return concat_final


final_df = formating_unifiedqa(concat_final)
final_df

## Save the output

In [ ]:
second_part = model_name.split('/')[1]
file_name = second_part + "_" + f"habitat.csv"

path = directory_name +  "/" + f"{second_part}"

if not os.path.exists(path):
    os.makedirs(path)
    print("Directory created:", path)
else:
    print("Directory already exists:", path)


full_path = f"{path}/{file_name}"

df_final.to_csv(full_path, index=False)

# Accuracy

In [ ]:
def accuracy(df):
    generated = df.iloc[:,2]
    groundtruth = df.iloc[:,1]

    return accuracy_score(groundtruth, generated)

In [ ]:
blue_bar = (.52, 0.44, 0.53, .44, .54, .52, .56, .41, .55, .53, .57)


llms = ['Llama-2-7b', 'Llama-2-13b', 'Llama-2-70b', 'Llama-3-8b','Llama-3-70b',
        'Flan-base', 'Flan-small', 'Flan-large', 'Flan-xl', 'Flan_alpaca_gpt4-xl',
       'Unifiedqa-v2']


fig = plt.figure(figsize = (10, 5))


plt.bar(llms, blue_bar, color ='blue', width = 0.3)

plt.xlabel("Model", fontdict={'fontsize': 20})
plt.ylabel("Accuracy", fontdict={'fontsize': 20})
plt.title("Habitat-centering", fontdict={'fontsize': 20})
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('pathtosave', dpi=300)
plt.show()
